# 3 Boosting Methods

In [9]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics

In [10]:
# import and split data 
train = pd.read_csv('/Users/jerry/Desktop/udemy courses/Kaggle数据分析/Avito Duplicate Ads Detection/train.csv')
X = train.drop(['Unnamed: 0', 'isDuplicate'],axis=1)
y = train[['isDuplicate']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

Normalization is used to reduce overfitting but since the result is not good, might not need this now. 

changing pandas dataframe to numpy array
X_train = X_train.to_numpy()
y_train = y_train.to_numpy()

X_test = X_test.to_numpy()
y_test = y_test.to_numpy()

normalizing the data
sc = StandardScaler()
X_train = sc.fit_transform(X_train)

X_test = sc.fit_transform(X_test)

## 3.1 Gradient Boosting Machines (GBM)

In [57]:
lr_list = [0.05, 0.075, 0.1, 0.25, 0.5, 0.75, 1]

for learning_rate in lr_list:
    gb_clf = GradientBoostingClassifier(n_estimators=20, learning_rate=learning_rate, max_features=2, max_depth=2, random_state=0)
    gb_clf.fit(X_train, y_train)

    print("Learning rate: ", learning_rate)
    print("Accuracy score (training): {0:.3f}".format(gb_clf.score(X_train, y_train)))
    print("Accuracy score (validation): {0:.3f}".format(gb_clf.score(X_test, y_test)))

/Users/jerry/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Learning rate:  0.05
Accuracy score (training): 0.675
Accuracy score (validation): 0.676


/Users/jerry/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Learning rate:  0.075
Accuracy score (training): 0.681
Accuracy score (validation): 0.681


/Users/jerry/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


...

Learning rate:  0.5
Accuracy score (training): 0.707
Accuracy score (validation): 0.706


/Users/jerry/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Learning rate:  0.75
Accuracy score (training): 0.705
Accuracy score (validation): 0.704


/Users/jerry/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Learning rate:  1
Accuracy score (training): 0.708
Accuracy score (validation): 0.708


In [58]:
# learning_rate=0.5 has the best result from above
gb_clf2 = GradientBoostingClassifier(n_estimators=20, learning_rate=0.5, max_features=2, max_depth=2, random_state=0)
gb_clf2.fit(X_train, y_train)
predictions = gb_clf2.predict(X_test)

print("Confusion Matrix:")
print(confusion_matrix(y_test, predictions))

print("Classification Report")
print(classification_report(y_test, predictions))

/Users/jerry/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Confusion Matrix:
[[242244  70136]
 [ 85662 132342]]
Classification Report
              precision    recall  f1-score   support

           0       0.74      0.78      0.76    312380
           1       0.65      0.61      0.63    218004

    accuracy                           0.71    530384
   macro avg       0.70      0.69      0.69    530384
weighted avg       0.70      0.71      0.70    530384



In [61]:
predictions_1 = gb_clf2.predict_proba(X_test)[:,1]

In [62]:
fpr0, tpr0, thresholds0 = metrics.roc_curve(y_test, predictions_1)
metrics.auc(fpr0, tpr0)

0.774474968406767

## Stochastic Gradient Boosting

##   AdaBoost

Increase weight for the good predictors. 

In [3]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from numpy import mean
from numpy import std
from matplotlib import pyplot

In [91]:
# get a list of models to evaluate
def get_models():
    models = dict()
    # define number of trees to consider
    n_trees = [10, 50, 100, 500, 1000]
    for n in n_trees:
        models[str(n)] = AdaBoostClassifier(n_estimators=n)
    return models

In [92]:
# evaluate a given model using cross-validation
def evaluate_model(model, X, y):
    # define the evaluation procedure
    # https://medium.com/@venkatasujit272/overview-of-cross-validation-3785d5414ece
    cv = RepeatedStratifiedKFold(n_splits=2, n_repeats=2, random_state=1) 
    # evaluate the model and collect the results
    scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
    return scores

In [93]:
models = get_models()
results, names = list(), list()

for name, model in models.items():
    # evaluate the model
    scores = evaluate_model(model, X, y)
    # store the results
    results.append(scores)
    names.append(name)
    # summarize the performance along the way
    print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))

>10 0.701 (0.000)
>50 0.706 (0.000)
>100 0.707 (0.000)
>500 0.710 (0.000)
>1000 0.712 (0.000)


In [ ]:
pyplot.boxplot(results, labels=names, showmeans=True)
pyplot.show()

In [5]:
ada = AdaBoostClassifier(n_estimators=1000).fit(X_train, y_train)

/Users/jerry/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


So when number of tree = 1000 model returns the best result. 

In [6]:
ada_predict = ada.predict_proba(X_test)[:,1]

In [7]:
fpr0, tpr0, thresholds0 = metrics.roc_curve(y_test, ada_predict)
metrics.auc(fpr0, tpr0)

0.7843416264428136

## LightGBM

## XGBoosting

In [11]:
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

/Users/jerry/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/Users/jerry/anaconda3/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [45]:
folds = 2 # splits/folds 
param_comb = 2 # different combinations

parameter_grid = {"learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
 "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
 "min_child_weight" : [ 1, 3, 5, 7 ],
 "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ] }

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

In [47]:
xgb_clf = XGBClassifier()
#xgb_clf.fit(X_train, y_train)

In [48]:
random_search = RandomizedSearchCV(xgb_clf, param_distributions=parameter_grid, 
                                   n_iter=param_comb, scoring='roc_auc', 
                                   n_jobs=-1, cv=skf.split(X_train, y_train), 
                                   verbose=3, random_state=1001 )

In [49]:
random_search.fit(X_train, y_train)

Fitting 2 folds for each of 2 candidates, totalling 4 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:  1.5min finished
/Users/jerry/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


RandomizedSearchCV(cv=<generator object _BaseKFold.split at 0x7fe492f557c8>,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, missing...
                                           scale_pos_weight=None,
                                           subsample=None, tree_method=None,
                                           validate_parameters=None,
                                           verbosity=

In [50]:
pd.DataFrame(random_search.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_min_child_weight,param_max_depth,param_learning_rate,param_gamma,param_colsample_bytree,params,split0_test_score,split1_test_score,mean_test_score,std_test_score,rank_test_score
0,82.725199,0.191587,4.168749,0.037452,3,6,0.1,0,0.7,"{'min_child_weight': 3, 'max_depth': 6, 'learn...",0.799263,0.799778,0.799521,0.000258,1
1,70.864484,0.095244,4.034479,0.048454,3,5,0.15,0,0.7,"{'min_child_weight': 3, 'max_depth': 5, 'learn...",0.798379,0.798528,0.798454,0.000074,2


In [51]:
random_search.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0.0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=3, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [52]:
random_search.score(X_test, y_test) #accuracy measure

0.7991623120556509

In [53]:
predictions1 = random_search.predict(X_test)

print("Confusion Matrix:")
print(confusion_matrix(y_test, predictions1))

print("Classification Report")
print(classification_report(y_test, predictions1))

Confusion Matrix:
[[245469  66911]
 [ 77783 140221]]
Classification Report
              precision    recall  f1-score   support

           0       0.76      0.79      0.77    312380
           1       0.68      0.64      0.66    218004

    accuracy                           0.73    530384
   macro avg       0.72      0.71      0.72    530384
weighted avg       0.73      0.73      0.73    530384



In [54]:
predictions2 = random_search.predict_proba(X_test)[:,1]

In [55]:
fpr0, tpr0, thresholds0 = metrics.roc_curve(y_test, predictions2)
metrics.auc(fpr0, tpr0)

0.7991623120556509

In [13]:
model = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0.0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=3, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [14]:
model1 = model.fit(X_train, y_train)

/Users/jerry/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [16]:
test = pd.read_csv('/Users/jerry/Desktop/udemy courses/Kaggle数据分析/Avito Duplicate Ads Detection/test.csv')
X1 = test.drop(['Unnamed: 0','index'],axis=1)
#X1 = sc.fit_transform(X1)
y_pred1 = model1.predict_proba(X1)

In [35]:
pred_test1 = y_pred1[:,1]
index = np.array(test['index'].astype(np.int))
pred_test2 = {'id':index,'probability':pred_test1}

In [40]:
pred_test3 = pd.DataFrame(pred_test2, columns =['id','probability'])

In [41]:
pred_test3.to_csv('/Users/jerry/Desktop/udemy courses/Kaggle数据分析/Avito Duplicate Ads Detection/prediction_xgboost.csv',index=False)

0.76184